In [ ]:
model_str = "GINO"  # or "GNN"

import matplotlib.pyplot as plt
from data_loading import load_dataset
import torch
from deepcardio.losses import LpLoss
from predict import ModelInference
import numpy as np

DATA_DIR = r"/mnt/research/compbiolab/Ehsan/DeepCardioSim/cardiac_models/electrophysio/data_processed"

if model_str == "GINO":
    from GINO.model import initialize_GINO_model
    model = initialize_GINO_model(n_fno_modes=16)
    folder_path=DATA_DIR + '/data_GINO.pt'
    from deepcardio.meshdata import BipartiteData
    dataset_format=BipartiteData
    save_dir = "./GINO/ckpt/"
    dataprocessor_path = "./GINO/data_processor.pt"
    from GINO.gino_data_handling import single_case_handling

elif model_str == "GNN":
    from GNN.model import initialize_GNN_model
    model = initialize_GNN_model(size_hidden_layers=16)
    folder_path=DATA_DIR + '/data_GNN.pt'
    from torch_geometric.data import Data
    dataset_format=Data
    save_dir = "./GNN/ckpt/ckpt_64/"
    dataprocessor_path = "./GNN/data_processor.pt"
    from GNN.gnn_data_handling import single_case_handling

else:
    raise ValueError("Only 'GINO' or 'GNN' can be passed.")

dltrain, dltest, data_processor = load_dataset(
    model=model_str,
    folder_path=folder_path,
    train_batch_sizes=[1], test_batch_sizes=[1, 1],
    use_distributed=False, dataset_format=dataset_format,
    dataprocessor_dir='./')

model_inference = ModelInference(
    model=model,
    model_checkpoint_path=save_dir + 'best_model_snapshot_dict.pt',
    dataprocessor_path=dataprocessor_path,
    single_case_handling=single_case_handling)


l2loss = LpLoss(d=2, p=2, reductions='mean')

train_loss = l2loss
eval_losses={'l2': l2loss}

print(f"EPOCH: {model_inference.current_epoch}, LOSS: {model_inference.best_loss}")
model_inference.data_processor.training, model_inference.model.training

In [ ]:
from deepcardio.neuralop_core.utils import count_model_params
count_model_params(model)

In [ ]:
import json
from pathlib import Path
with open(Path(save_dir).joinpath('metrics_dict.json').as_posix(), 'r') as f:
    list_epoch_metrics = json.load(f)

epochs = []
training_losses = []
test_losses = []

for metrics_data in list_epoch_metrics:
    epochs.append(metrics_data['epoch'])
    training_losses.append(metrics_data['train_err'])
    test_losses.append(metrics_data['0_l2'])
    
plt.plot(epochs, training_losses, label="Training")
plt.plot(epochs, test_losses, label="Validation")
plt.yscale("log")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()

In [ ]:
import pandas as pd
df = pd.DataFrame(columns=['case_id', 'loss'])
for i, sample in enumerate(dltrain[0]):
    # if sample['n_pacings'] == 1:
    #     continue
    model_inference.predict(sample)
    output = model_inference.output.to('cuda')
    train_loss = l2loss(output, **sample).item()
    torch.cuda.empty_cache()
    df.loc[i] = [int(sample['label'][0]), train_loss]

df.to_csv(f'train_losses.csv', index='case_id')

plt.plot(df['loss'])
plt.show()

In [ ]:
print(torch.tensor(df['loss']).mean())
torch.topk(torch.tensor(df['loss']), k=20)

In [ ]:
df['loss'].idxmax()

In [ ]:
# sample = dltrain[0].dataset[6283]
sample['label'], sample['n_pacings'], sample['y'].max(), l2loss(model_inference.predict(sample), **sample).item()

In [ ]:
meshfile = '/mnt/research/compbiolab/Ehsan/DeepCardioSim/cardiac_models/electrophysio/data/mesh/case'
xdmffile = './GINO/results/xdmf/case'

model_inference.predict('/mnt/research/compbiolab/Ehsan/DeepCardioSim/cardiac_models/electrophysio/data/mesh/case6283.vtk')
model_inference.case_ID = model_inference.case_ID[:-4]
model_inference.write_xdmf(
    mesh_directory=meshfile,
    xdmf_directory=xdmffile)

In [ ]:
val_losses = []
for sample in dltest[0]:
    model_inference.predict(sample)
    output = model_inference.output.to('cuda')
    test_loss = l2loss(output, **sample).item()
    torch.cuda.empty_cache()
    val_losses.append(test_loss)
plt.plot(val_losses)
plt.show()


In [ ]:
print(torch.tensor(val_losses).mean())
torch.topk(torch.tensor(val_losses), k=5)

In [ ]:
test_losses = np.zeros((len(dltest[1]), 9))  # d_iso, n_pacings, 0.0, 0.01, 0.02, 0.05, 0.1, 0.2, refined

In [ ]:
noise = 0.00
for i, sample in enumerate(dltest[1]):
    sample['input_geom'] += torch.randn_like(sample['input_geom']) * (noise * sample['input_geom'].std())
    f0 = sample['a'][:, :, 2:5]
    sample['a'][:, :, 2:5] += torch.randn_like(f0) * (noise * f0.std())
    output = model_inference.predict(sample)
    test_loss = l2loss(output, **sample).item()
    d_iso = sample['a'][:, 0, 1].mean().item()
    test_losses[i, 0] = d_iso
    test_losses[i, 1] = sample['n_pacings']
    test_losses[i, 2] = test_loss
    torch.cuda.empty_cache()
plt.plot(test_losses[:, 2])
plt.show()

In [ ]:
print(torch.tensor(test_losses[:, 2]).mean())
torch.topk(torch.tensor(test_losses[:, 2]), k=20)

In [ ]:
sample = dltest[1].dataset[5644]
sample['label'], sample['n_pacings'], sample['y'].max(), l2loss(model_inference.predict(sample), **sample).item()

In [ ]:
model_inference.write_xdmf()

In [ ]:
np.percentile(test_losses[:, 2], 99.5)

In [ ]:
import pandas as pd
df = pd.DataFrame(columns=['case_id', 'loss'])
for i, loss in enumerate(test_losses[:, 2]):
    df.loc[i] = [int(dltest[1].dataset[i]['label']), loss]

df.to_csv(f'test_losses.csv', index='case_id')
# np.save("/mnt/home/naghavis/Documents/Research/DeepCardioSim/cardiac_models/electrophysio/data/posttraining/test_GINO.npy", test_losses)

In [ ]:
import glob
import os
folder_path = '/mnt/research/compbiolab/Ehsan/DeepCardioSim/cardiac_models/electrophysio/data_realLV/HFC/mesh'
YHC_losses = []
vtk_files = []
vtk_files.extend(glob.glob(os.path.join(folder_path, '*.vtk')))

for i, file in enumerate(vtk_files):
    model_inference.predict(file)
    output = model_inference.output.to('cuda')
    model_inference.case_ID = model_inference.case_ID[:-4]
    interpolated_loss = l2loss(output, **model_inference.sample).item()
    torch.cuda.empty_cache()
    YHC_losses.append(interpolated_loss)
    
plt.plot(YHC_losses)
plt.show()

In [ ]:
print(torch.tensor(YHC_losses).mean())
torch.topk(torch.tensor(YHC_losses), k=20)

In [ ]:
vtk_files[33]

In [ ]:
model_inference.predict(vtk_files[33])
model_inference.case_ID = model_inference.case_ID[:-4]
model_inference.write_xdmf(
    mesh_directory='/mnt/research/compbiolab/Ehsan/DeepCardioSim/cardiac_models/electrophysio/data_realLV/HFC/mesh/case',
    xdmf_directory='./GINO/results/xdmf/case')

In [ ]:
model_inference.predict('/mnt/home/naghavis/Documents/Research/heArt/generate_EP_dataset/data/mesh/case177.vtk')
l2loss(model_inference.output, **model_inference.sample).item(), model_inference.case_ID

In [ ]:
model_inference.case_ID = model_inference.case_ID[:-4]
model_inference.write_xdmf(
    mesh_directory='/mnt/home/naghavis/Documents/Research/heArt/generate_EP_dataset/data/mesh/case',
    xdmf_directory='/mnt/home/naghavis/Documents/Research/DeepCardioSim/real_geom_res/case'
)

In [ ]:
len('_refinedinterp.vtk')

In [ ]:
dltest[1].dataset[0]

In [ ]:
import glob
import os
folder_path = './data/interpolated'
interpolated_losses = []
vtk_files = []
vtk_files.extend(glob.glob(os.path.join(folder_path, '*.vtk')))

meshes = []
# for i, file in enumerate(vtk_files):
    # model_inference.predict(file)
    # output = model_inference.output.to('cuda')
    # model_inference.case_ID = model_inference.case_ID[:-4]
    # interpolated_loss = l2loss(output, **model_inference.sample).item()
    # torch.cuda.empty_cache()
    # interpolated_losses.append(interpolated_loss)
    
# plt.plot(interpolated_losses)
# plt.show()

d_iso_dev_1ploc = []
d_iso_dev_2ploc = []
for i, file in enumerate(vtk_files):
    model_inference.predict(file)
    for data in dltest[1]:
        if data['label'] == model_inference.sample['label'][:-18]:
            n_pacings = data['n_pacings']
            break
    d_iso = model_inference.sample['a'][:, 0, 1].cpu().numpy().mean()
    max_tact = model_inference.sample['y'].max().cpu().numpy()
    if n_pacings == 1:
        d_iso_dev_1ploc.append(d_iso)
    elif n_pacings == 2:
        d_iso_dev_2ploc.append(d_iso)
np.save('./data_posttraining/d_iso_interp_1ploc.npy', d_iso_dev_1ploc)
np.save('./data_posttraining/d_iso_interp_2ploc.npy', d_iso_dev_2ploc)

In [ ]:
print(torch.tensor(interpolated_losses).mean())
torch.topk(torch.tensor(interpolated_losses), k=5)

In [ ]:
np.save("./data_posttraining/interp_GINO.npy", interpolated_losses)

In [ ]:
torch.tensor(test_losses).mean()

In [ ]:
torch.topk(torch.tensor(test_losses), k=20)

In [ ]:
# import pandas as pd

# df = pd.DataFrame(columns=['Loss', 'd_iso', 'n_pacings', 'max_tact'])
d_iso_dev_1ploc = []
d_iso_dev_2ploc = []
for sample in dltrain[0]:
    # loss = l2loss(model_inference.predict(sample), **sample).item()
    n_pacings = sample['n_pacings'].cpu().numpy()
    d_iso = sample['a'][:, 0, 1].cpu().numpy().mean()
    max_tact = sample['y'].max().cpu().numpy()
    if n_pacings == 1:
        d_iso_dev_1ploc.append(d_iso)
    elif n_pacings == 2:
        d_iso_dev_2ploc.append(d_iso)
    # df.loc[len(df)] = [loss, d_iso, n_pacings, max_tact]
np.save('./data_posttraining/d_iso_train_1ploc.npy', d_iso_dev_1ploc)
np.save('./data_posttraining/d_iso_train_2ploc.npy', d_iso_dev_2ploc)
# df.to_pickle('./data_posttraining/test_GINO_withoutnoise.pkl')

In [ ]:
i = -1
for sample in dltest[1]:
    i += 1
    if sample['n_pacings'] == 2 or sample['y'].max() < 120 or sample['y'].max() > 150:
        continue
    print(i, sample['label'], sample['n_pacings'], sample['y'].max(), l2loss(model_inference.predict(sample), **sample).item())

In [ ]:
# Inverse problem
import meshio
def optimization_function_wrapper(x):
    """
    This wraps the function for the inverse modeling
    optimization problem.

    Input:
    - x: a numpy array represents [D_iso, x_l, x_c]

    Output:
    - loss: The loss of the optimization problem, returned as a
    scalar numpy value
    """
    ploc_pmtr = [x[1], x[2], 1]
    geometry = meshio.read('./data/geometry' + sample['label'] + '.vtk')
    pmtr_coord = np.concatenate((
        geometry.point_data['x_l'],
        geometry.point_data['x_c'],
        geometry.point_data['x_t']), axis=1)

    min_loc = np.argmin(np.linalg.norm(pmtr_coord - np.array(ploc_pmtr), axis=1))
    ploc_xyz = geometry.points[min_loc].reshape((1, -1)) * 100


    output = model_inference.predict(sample, Diso=x[0], plocs=ploc_xyz)

    loss = l2loss(output, sample['y'])
    return loss.item()

from scipy.optimize import differential_evolution

bounds = [(0.1, 2.), (0.15, .8), (-3., 3.)]

result = differential_evolution(optimization_function_wrapper,
                                        bounds,
                                        seed=45,
                                        popsize=50,
                                        strategy='best1bin',
                                        maxiter=100,
                                        disp=True)

result.x

In [ ]:
# Ground truth
0.12689291957489515
[0.3247652436498567, -2.853311347773299]

In [ ]:
# CRT Optimization
def optimization_function_wrapper(x):
    """
    This wraps the function for the inverse modeling
    optimization problem.

    Input:
    - x: a numpy array represents [x_l2, x_c2]

    Output:
    - loss: The loss of the optimization problem, returned as a
    scalar numpy value
    """
    plocs_pmtr = [[result.x[1], result.x[2], 1], [x[0], x[1], 1]]
    geometry = meshio.read('./data/geometry' + sample['label'] + '.vtk')
    pmtr_coord = np.concatenate((
        geometry.point_data['x_l'],
        geometry.point_data['x_c'],
        geometry.point_data['x_t']), axis=1)
    
    plocs_xyz = []
    for ploc_ in plocs_pmtr:
        min_loc = np.argmin(np.linalg.norm(pmtr_coord - np.array(ploc_), axis=1))
        ploc_xyz = geometry.points[min_loc] * 100
        plocs_xyz.append(ploc_xyz)

    plocs_xyz = np.array(plocs_xyz)


    output = model_inference.predict(sample, Diso=result.x[0], plocs=plocs_xyz)

    return output.max().item()

bounds = [(0.15, .8), (-3., 3.)]

result_CRT = differential_evolution(
    optimization_function_wrapper,bounds,
    seed=45,
    popsize=50,
    strategy='best1bin',
    maxiter=100,
    disp=True)

result_CRT.x

In [ ]:
result_CRT.fun

In [ ]:
plocs_xyz = []

plocs_pmtr = [[result.x[1], result.x[2], 1], [result_CRT.x[0], result_CRT.x[1], 1]]
geometry = meshio.read('./data/geometry' + sample['label'] + '.vtk')
pmtr_coord = np.concatenate((
    geometry.point_data['x_l'],
    geometry.point_data['x_c'],
    geometry.point_data['x_t']), axis=1)

for ploc_ in plocs_pmtr:
    min_loc = np.argmin(np.linalg.norm(pmtr_coord - np.array(ploc_), axis=1))
    ploc_xyz = geometry.points[min_loc] * 100
    plocs_xyz.append(ploc_xyz)

plocs_xyz = np.array(plocs_xyz)

plocs_xyz

In [ ]:
ploc_pmtr = [result.x[1], result.x[2], 1]
geometry = meshio.read('./data/geometry' + sample['label'] + '.vtk')
pmtr_coord = np.concatenate((
    geometry.point_data['x_l'],
    geometry.point_data['x_c'],
    geometry.point_data['x_t']), axis=1)

min_loc = np.argmin(np.linalg.norm(pmtr_coord - np.array(ploc_pmtr), axis=1))
ploc_xyz = geometry.points[min_loc].reshape((1, -1)) * 100
ploc_xyz

In [ ]:
output = model_inference.predict(sample, Diso = result.x[0], plocs=plocs_xyz)

In [ ]:
output.max()

In [ ]:
i = 0
for sample in dltest[1]:
    if sample['label'] == ['35865']:
        print(i)
        break
    i+=1


In [ ]:
# 2701: 9709
# 3260: 36214
# 22: 26514 0.0108
# 37: 18362 0.0079

# 4170: 7963 0.0102

In [ ]:
sample = dltest[1].dataset[90]
sample['label'], sample['n_pacings'], sample['y'].max(), l2loss(model_inference.predict(sample), **sample).item()

In [ ]:
import meshio
def set_rand_pacingsites(sample, rand):
    np.random.seed(rand)
    x_l = np.random.uniform(0.15, 0.8)
    x_c = np.random.uniform(-3, 3)
    x_c_2 = np.random.uniform(-3, 3)
    plocs_pmtr = [[x_l, x_c, 1], [x_l, x_c_2, 1]]
    geometry = meshio.read('../data/geometry_case' + sample['label'] + '.vtk')
    pmtr_coord = np.concatenate((
        geometry.point_data['x_l'],
        geometry.point_data['x_c'],
        geometry.point_data['x_t']), axis=1)

    plocs_xyz = []
    for ploc_ in plocs_pmtr:
        min_loc = np.argmin(np.linalg.norm(pmtr_coord - np.array(ploc_), axis=1))
        ploc_xyz = geometry.points[min_loc] * 100
        plocs_xyz.append(ploc_xyz)

    plocs_xyz = np.array(plocs_xyz)

    new_sample = model_inference.set_pacingsite(sample, plocs_xyz)
    del new_sample['y']
    return new_sample


In [ ]:
t_act_min = 200
t_act_max = 10
for i in range(1000):
    new_sample = set_rand_pacingsites(sample, 1000 + i)
    output = model_inference.predict(new_sample)
    if output.max() > t_act_max:
        if output.max() > 131:
            continue
        t_act_max = output.max()
        model_inference.write_xdmf(
            inp=sample,
            mesh_directory='./data/mesh/case',
            xdmf_directory='./GINO/results/xdmf/maxcase')
    if output.max() < t_act_min:
        t_act_min = output.max()
        model_inference.write_xdmf(
            inp=sample,
            mesh_directory='./data/mesh/case',
            xdmf_directory='./GINO/results/xdmf/mincase')
        
    